# Differential Privacy for PRETSA

In [1]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 510, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 510 (delta 50), reused 57 (delta 24), pack-reused 424 (from 1)
Receiving objects: 100% (510/510), 11.51 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [2]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=58057a9241bb33a9c85a15ba81fcee510c9cefacc01f1a182c726fab1a75f6ef
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=c6dfe88d662a114ec5014ee99dccaa19f7d6471d770ea26574a7417366a0962b
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built anytree intervaltree


In [3]:
!python /content/PRETSA/dataset_collection.py

/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
Downloaded: /content/PRETSA/downloads/CoSeLoG.xes.gz
Processing CoSeLoG...
parsing log, completed traces :: 100% 1434/1434 [00:00<00:00, 3434.50it/s]
Save

## Running Annotations for three datasets

In [4]:
# Generates all the original annotation duration for each of the three datasets: CoSeLog, Sepsis and Road Traffc
# Generates /content/PRETSA/original_annotation/
!python /content/PRETSA/gc_add_annotations_duration.py

Processing: CoSeLoG
Writting: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Reading: /content/PRETSA/csv_outputs/CoSeLoG_dataset.csv
Processing: Sepsis
Writting: /content/PRETSA/original_annotation/Sepsis_duration.csv
Reading: /content/PRETSA/csv_outputs/Sepsis_dataset.csv


## Run Pretsa: Generates Pretsa Log


In [ ]:
t = 0.7

In [5]:
# Generates: /content/PRETSA/pretsalog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py t

 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Fixing K-Anonymity...
 Fixing T-closeness...
T-Closeness adjustments applied to 0 nodes.
 Modified 99 cases for k=4, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Fixing K-Anonymity...
 Fixing T-closeness...
T-Closeness adjustments applied to 0 nodes.
 Modified 112 cases for k=8, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k8_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k8_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Fixing K-Anonymity...
 Fixing T-closeness...
T-Closeness adjustments applied to 0 nodes.
 Modified 169 cases for

## Run Baseline: Generate Baseline log

In [1]:
# Generates: /content/PRETSA/baselinelogs/ Path(filePath).name.replace(".csv","_pretsa_baseline_k%s_t%s.csv" % (kString,tString))
# These contain the new annotations for duration after Running BASELINE method
# Should generate same content as baselogs zip directory

!python /content/PRETSA/gc_generate_baseline_log.py t

python3: can't open file '/content/PRETSA/gc_generate_baseline_log.py': [Errno 2] No such file or directory


## Calculate Statistics

In [8]:
#Generates: "baseline_event_logs_statistics/"
!python /content/PRETSA/calculateBaselineEventLogStatistics.py t

#Generates: "PRETSA_event_logs_statistics/"
!python /content/PRETSA/calculatePRETSAEventLogStatistics.py  t

{'Dataset': 'CoSeLoG', 'k': 4, 'method': 'pretsa', 'variants': 32, 'cases': 1434}
{'Dataset': 'CoSeLoG', 'k': 8, 'method': 'pretsa', 'variants': 27, 'cases': 1434}
{'Dataset': 'CoSeLoG', 'k': 16, 'method': 'pretsa', 'variants': 21, 'cases': 1434}
{'Dataset': 'Sepsis', 'k': 4, 'method': 'pretsa', 'variants': 216, 'cases': 1049}
{'Dataset': 'Sepsis', 'k': 8, 'method': 'pretsa', 'variants': 116, 'cases': 1049}
{'Dataset': 'Sepsis', 'k': 16, 'method': 'pretsa', 'variants': 60, 'cases': 1049}


## Generate Annotations

In [9]:
 #Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py

 #Generates dictPath + "pretsa_statistics_annotations.csv"
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py

#Generates original_annotations_pretsa.csv
!python /content/PRETSA/generateAnnotationOriginalDataset.py

## Calculate Annotation Errors

 Relative error of the annotations for each activity

In [10]:
#Needs: original_annotations_pretsa.csv, pretsa_baseline_statistics_annotations.csv, pretsa_statistics_annotations.csv
!python /content/PRETSA/calculateAnnotationError.py

4
8
16
4
8
16
